# 네이버뷰 크롤링 후 DB 적재

In [1]:
#모듈 import
from bs4 import BeautifulSoup
from selenium import webdriver
import time
import pandas  as pd 
import psycopg2
from datetime import date, timedelta
import re

In [2]:
# 자동 스크롤다운 함수
def scroll_down(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
    time.sleep(5)

#select문 만들기
def select_command(table) :
    command = 'select * from ' + table
    return command

#insert문 만들기
def insert_command(table) :
    command = 'insert into ' + table + ' (title, date, url, hashtag) values (%s, %s, %s, %s);'
    return command

'뷔스티에 원피스', '롱패딩', '숏패딩', '롱코트', '숏코트', '잠옷', '홈웨어', '트위드자켓', '블레이저 자켓', '싱글 자켓', '크롭 자켓', '레더 자켓', '루즈핏', '테일러드자켓', '린넨자켓', '반팔자켓', '라이더 자켓', '오버핏 자켓', '청자켓', '니트가디건', '볼레로', '아가일', '크롭가디건', '반팔가디건', '앙고라', '바람막이', '아노락', '윈드브레이커', '바시티자켓', '바시티점퍼', '야구점퍼', '여름아우터', '간절기아우터', '겨울아우터', '니트 맨투맨', '기모 맨투맨', '크롭 맨투맨', '남자 맨투맨', '반집업', '반팔 맨투맨', '크롭후드', '기모후드', '후드티', '남자 후드', '꾸안꾸', '플리스', '후리스', '반팔티', '크롭 반팔', '블라우스', '와이셔츠', '셔츠', '체크셔츠', '타이다이', '옥스퍼드셔츠', '남방', '슬리브티', '니트', '남자니트', '조끼', '크롭티', '봄슬랙스', '여름슬랙스', '가을슬랙스', '겨울슬랙스', '간절기슬랙스', '데님팬츠', '연청', '진청', '중청', '흑청', '그레이진', '와이드팬츠', '밴딩바지', '스키니진', '린넨팬츠', '반바지', '하이웨스트', '로우웨스트', '부츠컷', '치노팬츠', '트레이닝복', '트레이닝바지', '조거팬츠', '카고바지', '일자팬츠', '냉장고바지', '멜빵바지', '오버롤팬츠', '찢청', '미니스커트', '미디스커트', '롱스커트', '플레어스커트', 'A라인 스커트', 'H라인 스커트', '청치마', '테니스스커트', '프릴스커트', '레더 스커트', '랩스커트', '데님스커트', '플리츠스커트', '미니원피스', '미디원피스', '롱원피스', '나시원피스', '코르셋', '셔츠원피스', '벨벳원피스', '꽃원피스', '골지원피스', '골지티', '쉬폰원피스', '쉬폰블라우스', '린넨원피스', '카라원피스', '랩원피스', '니트원피스', '후드원피스', '코듀로이', '반팔원피스', '점프슈트', '봄룩북', '봄옷쇼핑', '봄코디', '봄옷하울', '여름룩북', '여름옷쇼핑', '여름코디', '여름하울', '가을룩북', '가을옷쇼핑', '가을코디', '가을하울', '겨울룩북', '겨울옷쇼핑', '겨울코디', '겨울하울', '캠퍼스룩', '아메카지룩', '프레피룩', '하객룩', '놈코어룩', '시티보이룩', '미니멀룩', '애슬레저룩', '데이트룩', '출근룩', '데일리룩'

keyword_list = ['무지주선반', '벽걸이선반', '조립형선반', '무볼트선반', '철제선반', '폴딩선반', '슈랙선반', '화분선반', '대나무선반', '무타공선반']
'침대', '매트리스', '토퍼', '소파', '테이블', '식탁', '책상', '거실장', 'tv장', '서랍', '수납장', '진열장', '책장', '선반', '행거', '옷장', '의자', '화장대', '콘솔','거울', '침구세트', '이불', '베개', '토퍼', '패드', '매트리스커버', '블라인드', '롤스크린', '커튼', '러그', '카페트', '매트', '쿠션', '방석', '소파커버', '조명', 'led평판등', '천장등', '장스탠드', '단스탠드', '데스크스탠드', '무드등', '장식조명', '벽조명', '센서등', '형광등', '집꾸미기', '방꾸미기', '인테리어', '티비다이', '템바보드', '오픈수납장', '혼수가구', '게이밍의자', '진열장', '장식장', '책꽂이', '와인셀러', '캐비닛', '식탁', '데스크', '콰트로 책상', '주방상판', '프래그마', '사이드책상', '베드테이블', '매쉬 의자', '사무용의자', '학생의자', '공부용의자', '중역의자', '스툴', '헤드형의자', '회전의자', '회의실의자', '팔걸이의자', '발매트', '욕실매트', '주방매트', '놀이매트', // '층간소음매트', '극세사매트', '현관매트', '소프트매트', '소음매트', '뽀송매트', '미끄럼방지매트', '사계절매트', '메모리폼', '플라스틱서랍', '이동형서랍', '와이드서랍', '시스템서랍', '전자렌지선반', '칸칸이서랍', '친환경서랍', '다용도서랍', '레일서랍', '투명서랍', '아쿠아소파', '패브릭소파', '가죽소파', '카우치', '천연소파', '방수소파', '리클라이너', '쿠션소파', '스웨이드소파', '자코모소파', '좌식소파', '워셔블소파', '이태리소파', '슈퍼싱글', '퀸사이즈침대', '킹사이즈침대', '일체형침대', '호텔침대', '스프링침대', '방수커버', '헤드리스침대', '딜라이트침대', '헹거', '왕자행거', '드레스옷장', '스탠드옷장', '선반형옷장', '거울옷장', '시스템옷장', '고정식옷장', '이동식옷장', '붙박이장', '슬라이딩옷장', '무볼트옷장', '거울화장대', '조명화장대', '전신화장대', '좌식화장대', '서랍화장대', '로마화장대', '콘센트화장대', '원목화장대', '레트로화장대'

keyword_list = ['책추천', '책리뷰', '책소개', '인생책', '필독서', '인문서적', '에세이', '여행 책', '라이트노벨 책', '자기계발 책']

# 포스팅 수 가져오기

In [11]:
keyword_list = ['화사 거울', '비정형거울', '아크릴거울', '셀레티거울']

In [12]:
driver = webdriver.Chrome('C:\chromedriver.exe')

for query_txt in keyword_list :

    #크롬드라이버 실행
    driver.get('http://www.naver.com')
    time.sleep(2)

    #키워드를 검색한 네이버창 접속
    element = driver.find_element_by_id("query")
    element.send_keys(query_txt)
    element.submit()

    # 뷰 클릭
    driver.find_element_by_link_text("VIEW").click()
    time.sleep(2)

    #필터 클릭
    driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
    time.sleep(2)

    #기간 1년 설정
    driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()

    time.sleep(3)

    no2 = [ ]           # 게시글 번호 컬럼
    title2 = [ ]        # 게시물 제목 컬럼
    bdate2 = [ ]        # 작성 일자 컬럼
    url2 = [ ]          # url 컬럼
    hashtag2 = [ ]      # 해시태그 컬럼

    no = 1

    #더이상 스크롤이 안되면 종료
    i = 1
    time.sleep(1)

    #현재 스크롤 위치를 last_height로 받아옴
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        #스크롤 한 번 실행 후 스크롤 위치를 new_height로 받아옴
        scroll_down(driver)
        new_height = driver.execute_script("return document.documentElement.scrollHeight")

        #더이상 스크롤이 안되면 종료
        if new_height == last_height:
            break

        last_height = new_height
        i += 1


    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    view_list = soup.find('ul','lst_total').find_all('li')
    print(query_txt, '포스트 개수 :', len(view_list))

    for i in view_list :
        
        no2.append(no)                            # 게시물 번호 리스트에 추가
        print(no, end=' ')

        all_title = i.find_all('a')
        title = all_title[5].get_text( )          # 게시물 제목
        title2.append(title)                      # 게시물 제목 리스트에 추가

        time.sleep(2)

        bdate = i.find('span','sub_time sub_txt').get_text( )  # 작성일자
        bdate.replace('.', '-')
        bdate2.append(bdate)                     # 작성일자 리스트에 추가

        time.sleep(2)

        url = str(all_title[0])
        url = url.split('"')[5]                 # 게시물 url
        url2.append(url)                        # url 리스트에 추가

        time.sleep(2)

        # 카페글은 hashtag가 없다면 댓글을 보여줌
        try :
            outline_test = i.find('div', 'total_tag_area').get_text()    
            
            # hashtag가 안달려있는 블로그 글도 있기때문에 오류 방지
            try:
                hashtag = " ".join(re.findall("#[가-힣]{1,}", outline_test))
                hashtag2.append(hashtag)
                    
            except Exception as e:
                hashtag = ""
                hashtag2.append(hashtag)

        except : 
            hashtag = ""
            hashtag2.append(hashtag)


        if no == 200 : #한번에 200개 까지만 크롤링 가능
            break
            
        no += 1

    print(query_txt, '끝')

    bdate3 = [] #전처리 후 작성일자

    # 작성일자를 날짜형식으로 통일화
    for day in bdate2 :

        if '시간 전' in day :       # n시간 전 업로드 글
            day = date.today()
        elif '분 전' in day :       # n분 전 업로드 글
            day = date.today()
        elif '일 전' in day :       # n일 전 업로드 글 
            day = date.today() - timedelta(int(day[0])) #8일전 글 부터는 날짜로 반환
        elif day == '어제' :        # 어제 업로드 글
            day = date.today() - timedelta(1)
        else :
            day = day.rstrip('.')
            day = day.replace('.', '-')
        
        bdate3.append(day)

    # df화
    naver_blog = pd.DataFrame()
    naver_blog['제목'] = title2
    naver_blog['작성일자'] = bdate3
    naver_blog['주소'] = url2
    naver_blog['해시태그'] = hashtag2

    #date-time 타입으로 변환
    naver_blog['작성일자'] = pd.to_datetime(naver_blog['작성일자'])

    print(naver_blog.tail())

    #postgresql 접속
    conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
    conn = psycopg2.connect(conn_string)
    cur = conn.cursor()

    table = 'view_interior'

    #table에 데이터 넣기
    for i in range(len(naver_blog)) :
        cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2], naver_blog.iloc[i][3]))
        conn.commit()

    #postgresql 연결종료
    conn.cursor().close()
    conn.close()

C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:\chromedriver.exe')
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:23: DeprecationW

화사 거울 포스트 개수 : 127
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 화사 거울 끝
                                               제목       작성일자  \
122  2021/11/10 수요일 20:33 / 시청: 294,097명/방송: 117개 2021-11-10   
123                           클림트 주얼리 상담 후기(계약전!) 2022-03-23   
124                       명상할때 눈치보여서 소리를 못지르겠어요.. 2022-06-05   
125                                       홍순학 연행가 2022-02-21   
126                                 원형서랍거울문고리(완료) 2021-06-25   

                                                    주소 해시태그  
122               https://cafe.naver.com/pon2123/10205       
123        https://cafe.naver.com/makemywedding/327732       
12

C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


비정형거울 포스트 개수 : 90
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 비정형거울 끝
                       제목       작성일자  \
85         인테리어 끝내고 입주했어요 2021-07-18   
86              비정형거울 새제품 2021-10-15   
87  비정형 전신거울 (인테리어 용품) 6만 2022-06-14   
88              비정형거울 구해요 2022-04-02   
89        자라홈 비정형 인테리어 거울 2021-11-26   

                                              주소 해시태그  
85       https://cafe.naver.com/seosanmom/961071       
86        https://cafe.naver.com/2005mam/1421744       
87  https://cafe.naver.com/joonggonara/923221360       
88        https://cafe.naver.com/isajime/5268967       
89  https://cafe.naver.com/joonggonara/881612227       


C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


아크릴거울 포스트 개수 : 124
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 아크릴거울 끝
                           제목       작성일자  \
119  목공수업용/방과후학습용 안전거울 판매합니다. 2021-11-05   
120             아크릴거울 제작할수있는곳 2021-10-04   
121                유리거울 아크릴거울 2022-05-03   
122     아크릴거울 40cm×180cm드림완료요 2021-08-19   
123          원목 아크릴 안전 거울(완료) 2022-03-28   

                                                 주소  \
119  https://cafe.naver.com/woodworkermarket/257359   
120          https://cafe.naver.com/misinggo/913441   
121    https://cafe.naver.com/joonggonara/914662908   
122   https://cafe.naver.com/yeongjongdolove/672632   
123         https://cafe.naver.com/wandovenus

C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("query")
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:15: DeprecationWarning: find_element_by_link_text is deprecated. Please use find_element(by=By.LINK_TEXT, value=link_text) instead
  driver.find_element_by_link_text("VIEW").click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:19: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[1]/div/div[3]/a').click()
C:\Users\User\AppData\Local\Temp/ipykernel_17024/2412222373.py:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="snb"]/div[2]/ul/li[3]/div/div[1]/a[8]').click()


셀레티거울 포스트 개수 : 98
1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 셀레티거울 끝
                                     제목       작성일자  \
93  대전 유성 힙감성ʕ•ᴥ•ʔ 수제버거 맛집! '버거베어 충남대점' 2022-04-10   
94                                   일상 2022-02-03   
95                     셀레티 토일렛페이퍼 립스틱거울 2021-06-17   
96            셀레티 토일렛 페이퍼 립스틱 거울 판매합니다. 2021-10-17   
97                        셀레티 토일렛페이퍼 거울 2021-09-28   

                                              주소  \
93   https://blog.naver.com/hee292h/222696800608   
94  https://blog.naver.com/huu26355/222637602998   
95  https://cafe.naver.com/joonggonara/848757494   
96  https://cafe.naver.com/joonggonara/872986060   
97  https://cafe.naver.com/joonggonara/868877773   

                                                 해시

#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

In [ ]:
#table 확인하기
cur.execute(select_command(table))
conn.commit()
cur.fetchall()

In [17]:
#db 연결 종료
conn.cursor().close()
conn.close()

In [23]:
#varchar(500) type에 안들어가면 실행
b = 0

for i in range(len(url2)) :
    a = len(url2[i])

    if a > b :
        b = a
        print(i, b)

print(title2[20])

0 44
2 45
4 46
8 101
9 103
87 211
88 216
플라스틱 수저분리함/ 300 400 500 600 900... 칸칸 싱크대 서랍정리 수저함 트레이 수납장 소개입니다.


In [24]:
#postgresql 접속
conn_string = "host='localhost' dbname='postgres' user='postgres' password='admin'"
conn = psycopg2.connect(conn_string)
cur = conn.cursor()

table = 'view_interior'

#table에 데이터 넣기
for i in range(len(naver_blog)) :
    cur.execute(insert_command(table), (naver_blog.iloc[i][0], naver_blog.iloc[i][1], naver_blog.iloc[i][2], naver_blog.iloc[i][3]))
    conn.commit()

#postgresql 연결종료
conn.cursor().close()
conn.close()